<a href="https://colab.research.google.com/github/suman111999/Breast-Cancer-Classification/blob/main/classifier_handling_unbalanced_by_SMOTEEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score  
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import itertools
from sklearn.metrics import f1_score,precision_score, recall_score, confusion_matrix, roc_curve, precision_recall_curve, accuracy_score,classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_excel('BreastCancer_Prognostic_v1.xlsx')
df.head()

,ID,Outcome,Time,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,Worst_perimeter,Worst_area,Worst_smoothness,Worst_compactness,Worst_concavity,Worst_concave_points,Worst_symmetry,Worst_fractal_dimension,Tumor_Size,Lymph_Node_Status
0,119513,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5
1,8423,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2
2,842517,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0
3,843483,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0
4,843584,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 35 columns):
ID                           198 non-null int64
Outcome                      198 non-null object
Time                         198 non-null int64
radius_mean                  198 non-null float64
texture_mean                 198 non-null float64
perimeter_mean               198 non-null float64
area_mean                    198 non-null float64
smoothness_mean              198 non-null float64
compactness_mean             198 non-null float64
concavity_mean               198 non-null float64
concave_points_mean          198 non-null float64
symmetry_mean                198 non-null float64
fractal_dimension_mean       198 non-null float64
radius_std_dev               198 non-null float64
texture_std_dev              198 non-null float64
perimeter_std_dev            198 non-null float64
area_std_dev                 198 non-null float64
smoothness_std_dev           198 non-null float64


In [ ]:
df.replace(to_replace='?',value=np.nan,inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 35 columns):
ID                           198 non-null int64
Outcome                      198 non-null object
Time                         198 non-null int64
radius_mean                  198 non-null float64
texture_mean                 198 non-null float64
perimeter_mean               198 non-null float64
area_mean                    198 non-null float64
smoothness_mean              198 non-null float64
compactness_mean             198 non-null float64
concavity_mean               198 non-null float64
concave_points_mean          198 non-null float64
symmetry_mean                198 non-null float64
fractal_dimension_mean       198 non-null float64
radius_std_dev               198 non-null float64
texture_std_dev              198 non-null float64
perimeter_std_dev            198 non-null float64
area_std_dev                 198 non-null float64
smoothness_std_dev           198 non-null float64


In [ ]:
df=df.fillna(df.mean())

In [ ]:
df.isna().values.any()

False

In [ ]:
df.drop('ID',axis=1,inplace=True)

In [ ]:
df.head()

,Outcome,Time,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,...,Worst_perimeter,Worst_area,Worst_smoothness,Worst_compactness,Worst_concavity,Worst_concave_points,Worst_symmetry,Worst_fractal_dimension,Tumor_Size,Lymph_Node_Status
0,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,0.07055,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5.0
1,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,0.14710,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2.0
2,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,0.08180,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0.0
3,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,0.10520,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0.0
4,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,0.10430,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0.0


In [ ]:
df['Outcome']=df['Outcome'].map({'R':1,'N':0})

In [ ]:
df['Outcome'].value_counts()#imbalanced

0    151
1     47
Name: Outcome, dtype: int64

In [ ]:
X=df.drop('Outcome',axis=1)
X.head()

,Time,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,Worst_perimeter,Worst_area,Worst_smoothness,Worst_compactness,Worst_concavity,Worst_concave_points,Worst_symmetry,Worst_fractal_dimension,Tumor_Size,Lymph_Node_Status
0,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,0.07055,0.1865,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5.0
1,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,0.14710,0.2419,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2.0
2,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,0.08180,0.2333,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0.0
3,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,0.10520,0.2597,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0.0
4,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,0.10430,0.1809,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0.0


In [ ]:
y=df['Outcome']
y.head()

0    0
1    0
2    0
3    0
4    1
Name: Outcome, dtype: int64

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

Using TensorFlow backend.


In [ ]:
from imblearn.combine import SMOTEENN

In [ ]:
sample=SMOTEENN(sampling_strategy=0.5)
X_new,y_new=sample.fit_resample(X,y)

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X_new,y_new,test_size=.3,random_state=0)

In [ ]:
print(X_train.shape)

(60, 33)


In [ ]:
lg=LogisticRegression(class_weight='balanced',random_state=0).fit(X_train, y_train)
print('logistic Regression training Accuracy:')
print(lg.score(X_train, y_train))
print('logistic Regression testing Accuracy:')
print(lg.score(X_test, y_test))
print()

logistic Regression training Accuracy:
0.7833333333333333
logistic Regression testing Accuracy:
0.7692307692307693



In [ ]:
y_pred=lg.predict(X_test)

In [ ]:
f1_score(y_test,y_pred)

0.5

In [ ]:
r= RandomForestClassifier(class_weight='balanced',random_state=0).fit(X_train, y_train)#overfitting
print('randomforest training Accuracy:')
print(r.score(X_train, y_train))
print('random forest training Accuracy:')
print(r.score(X_test, y_test))
    

randomforest training Accuracy:
1.0
random forest training Accuracy:
0.8076923076923077


In [ ]:
y_pred=r.predict(X_test)

In [ ]:
f1_score(y_test,y_pred)

0.4444444444444445